In [1]:
import sqlite3
filename = 'cbdata.sqlite'
conn = sqlite3.connect(filename)
cur = conn.cursor()
cur.executescript('''
CREATE TABLE IF NOT EXISTS Publications (id TEXT NOT NULL PRIMARY KEY UNIQUE, title TEXT);
CREATE TABLE IF NOT EXISTS Datasets (id TEXT NOT NULL PRIMARY KEY UNIQUE, title TEXT);
CREATE TABLE IF NOT EXISTS Referencing (id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE, publication_id INTEGER, dataset_id TEXT);
''')
conn.commit()

In [4]:
import pandas as pd
csvname = '../../datasets/osdr_references.csv'
df = pd.read_csv(csvname)

In [5]:
for index, data in df.iterrows():
  dataset_title = ''
  publication_title = data['paper_title']
  publication_id = data['paper_pmcid']
  dataset_id = data['dataset_accession']
  cur.execute('SELECT COUNT(*) FROM Datasets WHERE id=(?)', (dataset_id,))
  count = cur.fetchone()[0]
  if count == 0:
    cur.execute('INSERT INTO Datasets (id, title) VALUES (?, ?)', (dataset_id, dataset_title))
  cur.execute('SELECT COUNT(*) FROM Publications WHERE id=(?)', (publication_id,))
  count = cur.fetchone()[0]
  if count == 0:
    cur.execute('INSERT INTO Publications (id, title) VALUES (?, ?)', (publication_id, publication_title))
  cur.execute('INSERT INTO Referencing (publication_id, dataset_id) VALUES (?, ?)', (publication_id, dataset_id))
conn.commit()